In [3]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolo12n-seg.yaml")  # build a new model from YAML
model = YOLO("yolo12n-seg.yaml").load("yolo12n.pt")  # build from YAML and transfer weights

Transferred 691/753 items from pretrained weights


In [5]:
model2 = YOLO('yolo12n-seg.yaml')